In [1]:
from matplotlib import pyplot as plt
import pandas as pd
from collections import defaultdict
import numpy as np
import os
import itertools

In [2]:
dataname = "emailEnron_0"
ep=100
labelnum = 3

In [3]:
def read_confusion_matrices(dirpath, fname="log_test_confusion.txt", ep=10, labelnum=3):
    if os.path.isfile(dirpath + "log_valid_confusion.txt") is True:
        fname = "log_valid_confusion.txt"
        evalname = "valid"
    else:
        evalname = "test"
        
    if os.path.isfile(dirpath + fname) is False:
        # print("not exist", dirpath + fname)
        return -1
    if os.path.isfile(dirpath + "log_{}_micro.txt".format(evalname)):
        num_epoch = 0
        num_times = 0
        best_f1 = 0
        total_epoch = 0
        with open(dirpath + "log_{}_micro.txt".format(evalname), "r") as f:
            for i, line in enumerate(f.readlines()):
                ep_str = line.rstrip().split(":")[0]
                f1_str = line.rstrip().split(":")[-1]
                epoch = int(ep_str.split(" ")[0])
                f1 = float(f1_str)
                if best_f1 < f1:
                    num_epoch = epoch
                    num_times = i + 1
                    best_f1 = f1
                else:
                    total_epoch = epoch
    else:
        best_f1 = 0
        num_epoch = ep
        total_epoch = ep
        num_times = 1
            
    with open(dirpath + fname, "r") as f:
        one_input = []
        for line in f.readlines():
            if len(one_input) == labelnum:
                num_times -= 1
                if num_times == 0:
                    break
                one_input = []
            one_input.append(line)
                
    confusion_matrix = np.zeros((labelnum,labelnum))
    for i in range(labelnum):
        line = one_input[i].rstrip()
        tmp = line.split("\t")
        for c, v in enumerate(tmp):
            confusion_matrix[i,c] = int(v)

    accuracy = sum([confusion_matrix[i, i] for i in range(labelnum)]) / (np.sum(confusion_matrix))
    
    rec_bar = []
    label_num = []
    recall_micro_denom = 0
    recall_micro_numer = 0
    for i in range(confusion_matrix.shape[0]):
        row_sum = np.sum(confusion_matrix[i])
        label_num.append(row_sum)
        ans = confusion_matrix[i,i]
        if row_sum == 0:
            rec_bar.append(0)
        else:
            rec_bar.append(ans / row_sum)
        recall_micro_denom += row_sum
        recall_micro_numer += ans
    
    recall_macro = sum(rec_bar) / labelnum
    recall_micro = recall_micro_numer / recall_micro_denom
    
    prec_bar = []
    pred_label_num = []
    precision_micro_denom = 0
    precision_micro_numer = 0
    for i in range(confusion_matrix.shape[0]):
        col_sum = np.sum(confusion_matrix[:,i])
        pred_label_num.append(col_sum)
        ans = confusion_matrix[i,i]
        if col_sum == 0:
            prec_bar.append(0)
        else:
            prec_bar.append(ans / col_sum)
        precision_micro_denom += col_sum
        precision_micro_numer += ans
    precision_macro = sum(prec_bar) / labelnum
    precision_micro = precision_micro_numer / precision_micro_denom
    
    f1_bar = []
    for i in range(len(rec_bar)):
        if (prec_bar[i] + rec_bar[i]) == 0:
            f1_bar.append(0)
        else:
            f1_bar.append(2 * ( (prec_bar[i] * rec_bar[i]) / (prec_bar[i] + rec_bar[i]) ))
    f1score_macro = sum(f1_bar) / labelnum
    f1score_micro = 2 * ( (precision_micro * recall_micro) / (precision_micro + recall_micro))
    
    if f1score_macro == 1.00:
        print(confusion_matrix)
        print(rec_bar)
        print(prec_bar)
        print(f1_bar)
        print(dirpath)
    assert best_f1 == 0 or best_f1 == f1score_micro, dirpath + "\t" + str(best_f1) + " " + str(f1score_micro)
    
    # Label Diff
    numlabelrate = []
    total_label_ratio = 0
    all_labels = sum(label_num)
    for i in range(labelnum):
        numlabelrate.append((abs(pred_label_num[i] - label_num[i]) / label_num[i])) # pred_label_num[i] / label_num[i]) # distance??
        total_label_ratio += (abs(pred_label_num[i] - label_num[i]) / label_num[i]) * (label_num[i] / all_labels)
#         numlabelrate.append(1.0 - (abs(pred_label_num[i] - label_num[i]) / label_num[i])) # pred_label_num[i] / label_num[i]) # distance??
#         total_label_ratio += (abs(pred_label_num[i] - label_num[i]) / label_num[i])
    
    
    result = {
        "accuracy" : accuracy,
        "f1_micro": f1score_micro,
        "f1_macro": f1score_macro,
        "num_epoch" : num_epoch,
        "total_epoch" : total_epoch,
        "labeldiff" : total_label_ratio,
    }
    for i in range(labelnum):
        result["recall_" + str(i)] = rec_bar[i]
        result["precision_" + str(i)] = prec_bar[i]
        result["f1_" + str(i)] = f1_bar[i]
        result["labelratio_" + str(i)] = numlabelrate[i]
    return result

In [4]:
resultdict = defaultdict(dict)
model2bestparam = {}

In [5]:
# Add F1 macro
for initvec in ["rw"]: # "onehot", "adj",
    orgdir = "../results_v2/" + dataname + "/" + initvec + "/"
    
    refined_list = defaultdict(list)
    for modelname in os.listdir(orgdir):
        modelnamedir = orgdir + modelname + "/"
#         print(modelnamedir)
        if "past" in modelname:
            continue
        if os.path.isdir(modelnamedir) is False:
            continue
        embedder = modelname.split("_")[0]
        if len(embedder.split("-")) > 1:
            _, attV, aggV, attE, aggE = embedder.split("-")
            if attE != "pure" and aggE == "PrevQ": # Common Condition!
                refined_list[embedder].append(modelnamedir)
    
    model2param = defaultdict(dict)
    for modelname in refined_list:
        for modelnamedir in refined_list[modelname]: # number of layer
#             print(modelnamedir)
            for paramname in os.listdir(modelnamedir): # parameters   
                # get Result
                paramnamedir = modelnamedir + paramname + "/"
                if os.path.isdir(paramnamedir) is False:
                    continue
                res = read_confusion_matrices(paramnamedir, labelnum=labelnum)
                if res != -1:
                    print(paramnamedir)
                    if modelname not in model2param:
                        model2param[modelname] = {}
                    f1 = (res["f1_micro"] + res["f1_macro"]) / 2
                    if abs(res["f1_micro"]-res["f1_macro"]) > 0.2:
                        continue
                    if paramnamedir not in model2param[modelname]:
                        model2param[modelname][paramnamedir] = f1
                    elif model2param[modelname][paramnamedir] < f1:
                        model2param[modelname][paramnamedir] = f1

    for modelname in model2param.keys():
        param2f1 = model2param[modelname]
        if len(param2f1) == 0:
            print(modelname_w_param)
            continue
        sorted_param = sorted(list(param2f1.keys()), key=lambda x: param2f1[x], reverse=True)
        for paramkey in sorted_param:
            res = read_confusion_matrices(paramkey, ep=ep, labelnum=labelnum)
            if res != -1:
                res["searchspace"] = len(param2f1)
                
                paramname = paramkey.split("/")[-2]
                pe = "-"
                if paramname.split("_")[-2] == "pe":
                    pe = paramname.split("_")[-1]
                res["pe"] = pe
                
                modeldir = paramkey.split("/")[-3]
                _, atnl, nl, _, _ = modeldir.split("_")
                atnl = int(atnl[-1])
                res["atnl"] = atnl
                
                
                
                if modelname not in resultdict:
                    resultdict[modelname] = res
                    model2bestparam[modelname] = paramkey

../results_v2/emailEnron_0/rw/transformer-RankAdd-PrevQ-RankAdd-PrevQ_atnl1_nl1_sm_snl1/hd_64_od_128_bs_64_lr_0.001_ni_4_sp_40/


In [6]:
print(resultdict)

defaultdict(<class 'dict'>, {'transformer-RankAdd-PrevQ-RankAdd-PrevQ': {'accuracy': 0.8196749439151715, 'f1_micro': 0.8196749439151715, 'f1_macro': 0.7622133470792942, 'num_epoch': 65, 'total_epoch': 50, 'labeldiff': 0.032351187539945696, 'recall_0': 0.8470554351043659, 'precision_0': 0.8734058174258797, 'f1_0': 0.8600288365859107, 'labelratio_0': 0.030169689502612146, 'recall_1': 0.8215467916415465, 'precision_1': 0.8042381657628365, 'f1_1': 0.8128003421227366, 'labelratio_1': 0.021521766331858358, 'recall_2': 0.6423238566131027, 'precision_2': 0.5877216793340572, 'f1_2': 0.6138108625292353, 'labelratio_2': 0.09290482076637825, 'searchspace': 1, 'pe': '-', 'atnl': 1}})


In [7]:
# if dataname != "emailEu_0":
evallist = ["f1_micro", "f1_macro"]
# else:
#     evallist = ["f1_micro", "f1_macro"] 

In [8]:
outputname = "PEResult_{}.txt".format(dataname)
with open(outputname, "w") as f:
    line = "Model,AttV,AggV,AttE,AggE,atnl,pe," + ",".join(evallist) + ",searchspace,epoch"
    f.write(line + "\n")

In [9]:
for k in resultdict:
    modelname = k
    tmp = modelname.split("-")
    
    if len(tmp) == 1:
        model = tmp[0]
        attV, aggV, attE, aggE = "-", "-", "-", "-"
    else:
        model, attV, aggV, attE, aggE = tmp
    
    line = [model, attV, aggV, attE, aggE, str(resultdict[k]["atnl"]), str(resultdict[k]["pe"])] + [str(resultdict[k][e]) for e in evallist] + [str(resultdict[k]["searchspace"]), str(resultdict[k]["num_epoch"])]
    line = ",".join(line)
    with open(outputname, "a") as f:
        f.write(line + "\n")

In [10]:
print(dataname)
d = pd.read_csv(outputname)
# print(d)
d = d.sort_values(by=["f1_micro"], ascending=False)
d

emailEnron_0


,Model,AttV,AggV,AttE,AggE,atnl,pe,f1_micro,f1_macro,searchspace,epoch
0,transformer,RankAdd,PrevQ,RankAdd,PrevQ,1,-,0.819675,0.762213,1,65


In [11]:
for k, v in model2bestparam.items():
    print(k)
    print(v)
    print("--------------")

transformer-RankQ-PrevQ-pure-PrevQ
../results_v2/DBLP2_0/rw/transformer-RankQ-PrevQ-pure-PrevQ_atnl1_nl2_sm_snl1/hd_64_od_128_bs_64_lr_0.001_ni_4_sp_-1/
--------------
transformer-ShawRE-PrevQ-pure-PrevQ
../results_v2/DBLP2_0/rw/transformer-ShawRE-PrevQ-pure-PrevQ_atnl1_nl1_sm_snl1/hd_64_od_128_bs_32_lr_0.001_ni_4_sp_-1_pe_KPRW/
--------------
transformer-ITRE-PrevQ-pure-PrevQ
../results_v2/DBLP2_0/rw/transformer-ITRE-PrevQ-pure-PrevQ_atnl1_nl1_sm_snl1/hd_64_od_128_bs_32_lr_0.001_ni_4_sp_-1_pe_KD/
--------------
transformer-RankAdd-PrevQ-pure-PrevQ
../results_v2/DBLP2_0/rw/transformer-RankAdd-PrevQ-pure-PrevQ_atnl1_nl2_sm_snl1/hd_64_od_128_bs_64_lr_0.001_ni_4_sp_-1/
--------------
transformer-pure-PrevQ-pure-PrevQ
../results_v2/DBLP2_0/rw/transformer-pure-PrevQ-pure-PrevQ_atnl1_nl1_sm_snl1/hd_64_od_128_bs_64_lr_0.001_ni_4_sp_-1/
--------------
